# Importation and Initialisation

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# from firebase import FirebaseManager 
from mongo import MongoDBManager
from selenium.webdriver.common.keys import Keys
import os
from dotenv import load_dotenv
from tqdm.notebook import tqdm
from gridfs import GridFS
import pandas as pd

In [2]:
collection_names=["professions_by_field","posts_by_profession","posts_by_profession_by_language"]
# credential_path="./indeedrecommender-firebase.json"
# firebase_manager=FirebaseManager(credential_path)

In [3]:
load_dotenv()
uri = os.getenv('MONGODB_URI')
database_name = os.getenv('DATABASE_NAME')
mongo_manager = MongoDBManager(uri, database_name)
stats = mongo_manager.get_database_storage_infos()

Taille des données : 215.68148612976074 Mo
Taille des données en pourcentage: 42.125290259718895 %


# Retrieve Fields and theirs professions from Ulaval Website

In [ ]:
def retrieve_professions_from_field_first(field):
    """
    Retrieves professions from the field using the provided field parameter.

    Args:
        field (str): The field to retrieve professions from.

    Raises:
        Exception: If no elements are found for the first function for the given field.

    Returns:
        None
    """
    global datas
    h3_elements = driver.find_elements(By.XPATH, "//h3[contains(text(), 'Professions')]")
    print(h3_elements)
    if len(h3_elements)==0:
        raise Exception(f"Aucun element trouvé pour la première fonction pour {field}")
    for h3_element in h3_elements:
        ul_element = h3_element.find_element(By.XPATH, "following-sibling::ul")
        driver.execute_script("arguments[0].scrollIntoView(true);", h3_element)
        if ul_element:
            li_elements = ul_element.find_elements(By.XPATH, "./li")
            for element in li_elements:
                text = driver.execute_script("return arguments[0].innerText;", element)
                # print(text)
                if text not in datas[field]:
                    datas[field].append(text)
            # print(datas)
        time.sleep(0.5)
    # driver.back()



def retrieve_professions_from_field_second(field):
    """
    Retrieves professions from the second field.

    Args:
        field (str): The field to retrieve professions from.

    Raises:
        Exception: If no elements are found for the second function for the given field.

    Returns:
        None
    """
    global datas
    strong_elements = driver.find_elements(By.XPATH, "//p[strong[contains(text(), 'Professions')]]")
    print(f"strong: {strong_elements}")
    if len(strong_elements)==0:
        errors.append(field)
        raise Exception(f"Aucun element trouvé pour la deuxième fonction pour {field}")
    for strong_element in strong_elements:
        ul_element = strong_element.find_element(By.XPATH, "following-sibling::ul")
        if ul_element:
            driver.execute_script("arguments[0].scrollIntoView(true);", ul_element)
            li_elements = ul_element.find_elements(By.XPATH, "./li")
            for element in li_elements:
                text = driver.execute_script("return arguments[0].innerText;", element)
                # ##################print(text)
                if text not in datas[field]:
                    datas[field].append(text)
            # print(datas)
        time.sleep(0.5)
    # driver.back()

def retrieve_professions_from_field(field):
    global nb_of_functions_tried, nb_of_functions
    try:
        if nb_of_functions_tried==1:
            retrieve_professions_from_field_first(field)
            driver.back()
        if nb_of_functions_tried==2:
            retrieve_professions_from_field_second(field)
            driver.back()
            nb_of_functions_tried=1
    except Exception as e:
        print(e)
        nb_of_functions_tried+=1
        retrieve_professions_from_field(field)

    if nb_of_functions_tried > nb_of_functions:
        driver.back()
        nb_of_functions_tried=1
    return



In [ ]:
driver = webdriver.Chrome()
driver.get("https://www.ulaval.ca/etudes/programmes")
title = driver.title

# Keys represente fields and values are arrays whose contains professions 
datas={}
errors=[]
nb_of_functions=2
nb_of_functions_tried=1

pagination_next_element=driver.find_element(By.ID,"pagination-heading").find_element(By.XPATH, "following-sibling::ul").find_element(By.XPATH, "./li[@class='pagination-item pagination-item--suivant']")
print(pagination_next_element)
while pagination_next_element:
    results_element= driver.find_element(By.ID,"resultats")
    children=results_element.find_elements(By.XPATH, "./ul[@class='views-row']")

    for child in children:
        element=child.find_element(By.XPATH, "./li/a")
        field=element.find_element(By.XPATH,"./div[@class='main-info']/h3[@class='titre-programme']").text
        if field not in datas:
            datas[field]=[]     
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        time.sleep(0.5)
        driver.execute_script("arguments[0].click();", element)
        retrieve_professions_from_field(field)
    
    driver.execute_script("arguments[0].scrollIntoView(true);", pagination_next_element)
    time.sleep(0.5)
    driver.execute_script("arguments[0].click();", pagination_next_element.find_element(By.TAG_NAME, "a"))
    try:
        pagination_next_element=driver.find_element(By.ID,"pagination-heading").find_element(By.XPATH, "following-sibling::ul").find_element(By.XPATH, "./li[@class='pagination-item pagination-item--suivant']")
    except Exception as e:
        pagination_next_element=None
   
driver.close()
print(datas)


In [ ]:
for key in datas.keys():
    mongo_manager.add_data_to_mongodb(collection_names[0],key,{"professions":datas[key]})


# Retrieve positions from Indeed

In [22]:
import re
from datetime import datetime
from dateutil.relativedelta import relativedelta

def stateDateToDate(stateDate):
   """
   Converts a state date string into a datetime object.
   Parameters:
   stateDate (str): The state date string to be converted.
   Returns:
   datetime: The converted datetime object.
   """

   date=None
   value=None
   for str in stateDate.split(" "):
      try:
         date= datetime.strptime(str, '%Y-%m-%d')
      except Exception as e:
         date=None
      
      if date:
         return date
   try:
      match = re.search(r'\d+', stateDate)
      if match:
         value= int(match.group())
   except Exception as e:
      e

   if value:
      if bool(re.compile(r'\b(\w*jour\w*|\w*day\w*)\b', re.IGNORECASE).search(stateDate)):
         date= (datetime.now()-relativedelta(days=value)).strftime('%Y-%m-%d')
      if bool(re.compile(r'\b(\w*mois\w*|\w*month\w*)\b', re.IGNORECASE).search(stateDate)):
         date= (datetime.now()-relativedelta(months=value)).strftime('%Y-%m-%d')
      if bool(re.compile(r'\b(\w*an \w*|\w*year\w*|\w*année\w*)\b', re.IGNORECASE).search(stateDate)):
         date= (datetime.now()-relativedelta(years=value)).strftime('%Y-%m-%d')
      if bool(re.compile(r'\b(\w*now\w*|\w*maintenant\w*|\w*instant\w*)\b', re.IGNORECASE).search(stateDate)):
         date= (datetime.now()).strftime('%Y-%m-%d')            
   return date

In [23]:
pattern_salary_ca = re.compile(r'(?:De\s)?([\d\s]+)\s?\$(?:\s?à\s([\d\s]+)\s?\$\s?)(?:par\s|per\s|annuel\s|yearly\s)?(year|annum|an)\s*', re.IGNORECASE)
pattern_salary_us = re.compile(r'(?:From\s)?\$\d+(?:,\d{3})*(?:\.\d{2})?(?:–\$\d+(?:,\d{3})*(?:\.\d{2})?)?\s?(?:a|per)\s(hour|day|week|month|year)\s*', re.IGNORECASE)
pattern_salary_fr = re.compile(r'(?:De\s)?([\d\s]+)\s?€\s?à\s([\d\s]+)\s?€\s?(?:par\s|per\s|annuel\s|yearly\s|annum\s)?(an|year)\s*', re.IGNORECASE)
pattern_jobtype = re.compile(r'\s([A-Za-zéèàç\s?/]+(?:,\s[A-Za-zéèàç\s]+)*)', re.IGNORECASE)

In [24]:
positionsDetailsList = {}

def indeedPositionInfos(profession, country,language,page_lang):
    """
    Retrieves position details from Indeed job listings.
    Args:
        profession (str): The profession to search for.
    Returns:
        None
    Raises:
        None
    """
    global positionsDetailsList
    positionsDetails={}
    positionTitle=''
    companyName=''
    location=''
    jobType=''
    salaryInfoAndJobType=''
    jobBenefits=[]
    benefits=''
    shiftAndSchedule=''
    description=''
    positionPostedDate=''
    postedText=''
    salary=''
    try:
        positionsDescriptionList = driver.find_element(By.XPATH, "//div[@id='mosaic-provider-jobcards']")
        ulElement = positionsDescriptionList.find_element(By.XPATH, "./ul")
        liElements = ulElement.find_elements(By.XPATH, "./li[@class='css-5lfssm eu4oa1w0']")
    except Exception as e:
        liElements=[]
    for li in liElements:
        positionsDetails = {}
        driver.execute_script("arguments[0].scrollIntoView(true);", li)
        time.sleep(0.5)
        try:
          position=li.find_element(By.XPATH, ".//h2[contains(@class, 'jobTitle')]/a")
          positionTitle = driver.execute_script("return arguments[0].innerText;", position)
        #   print(positionTitle)
        except Exception as e:
            # print("No position found")
            position=None

        try:
          dateElement=li.find_element(By.XPATH, ".//span[@data-testid='myJobsStateDate']")
          postedText=driver.execute_script("return arguments[0].innerText.replace('Posted', '').trim().replace(/\\n/g, ' ');", dateElement)
          positionPostedDate=stateDateToDate(postedText)
        except Exception as e:
            # print("No Date found")
            dateElement=None
        
        if position :
            driver.execute_script("arguments[0].click();", position)
            time.sleep(0.5)
            try:
               positionsDetailsElement = driver.find_element(By.XPATH, "//div[@id='jobsearch-ViewjobPaneWrapper']")
            except Exception as e:
               positionsDetailsElement=None
            #    print("No position details found")
               
            
            if positionsDetailsElement:	
                try:
                    companyName = driver.execute_script("return arguments[0].innerText;",positionsDetailsElement.find_element(By.XPATH, ".//div[@data-testid='inlineHeader-companyName']"))
                except Exception as e:
                    # print("No company name found")
                    companyName=''
                try:
                    location = driver.execute_script("return arguments[0].innerText;",positionsDetailsElement.find_element(By.XPATH, ".//div[@data-testid='inlineHeader-companyLocation']"))
                except Exception as e:  
                    # print("No location found")
                    location=''
                try:
                    salaryInfoAndJobType = driver.execute_script("return arguments[0].innerText;",positionsDetailsElement.find_element(By.ID, "salaryInfoAndJobType"))
                    # print(F"JOB type: {jobType}")
                except Exception as e:
                    # print("No job type found")
                    salaryInfoAndJobType=''
                # try:
                #     jobType2 = driver.execute_script("return arguments[0].innerText;",positionsDetailsElement.find_element(By.ID,"jobDetailsSection").find_element(By.XPATH, ".//div[@data-testid='Permanent-tile']"))
                #     # print(F"JOB type 2: {jobType2}")
                # except Exception as e:
                #     # print("No job type 2 found")
                #     jobType2=""
                # try:
                #     jobType3 = driver.execute_script("return arguments[0].innerText;",positionsDetailsElement.find_element(By.XPATH, ".//div[@data-testid='Full-time-tile']"))
                #     # print(F"JOB type 3: {jobType3}")
                # except Exception as e:
                #     # print("No job type 3 found")
                #     jobType3=""
                # try:
                #     jobType4 = driver.execute_script("return arguments[0].innerText;",positionsDetailsElement.find_element(By.XPATH, ".//div[@data-testid='Part-time-tile']"))
                #     # print(F"JOB type 4: {jobType4}")
                # except Exception as e:
                #     # print("No job type 4 found")
                #     jobType4=""
                if country == "Canada": 
                    salary = re.search(pattern_salary_ca, salaryInfoAndJobType)
                if country == "US":
                    salary = re.search(pattern_salary_us, salaryInfoAndJobType)
                if country == "France" and re.search(pattern_salary_fr, salaryInfoAndJobType):
                    salary = re.search(pattern_salary_fr, salaryInfoAndJobType)
                
                if salary:
                    try:
                        salary = salary.group()
                    except Exception as e:
                        salary = salary
                else:
                    try:
                        if page_lang == "fr":
                            salary = driver.execute_script("return arguments[0].innerText;",positionsDetailsElement.find_element(By.XPATH,".//h3[contains(text(), 'Salaire')]").find_element(By.XPATH, "following-sibling::div"))
                        if page_lang == "en":
                            salary = driver.execute_script("return arguments[0].innerText;",positionsDetailsElement.find_element(By.XPATH,".//h3[contains(text(), 'Pay')]").find_element(By.XPATH, "following-sibling::div"))
                        # print(F"Shift and schedule: {shiftAndSchedule}")
                    except Exception as e:
                        # print("No shift and schedule found")
                        salary=''
                    try:
                        jobType = []
                        if page_lang == "fr":
                            children=positionsDetailsElement.find_element(By.XPATH,".//h3[contains(text(), 'Type de poste')]").find_element(By.XPATH, "following-sibling::div").find_elements(By.XPATH, "./ul/li")
                        if page_lang == "en":
                            children=positionsDetailsElement.find_element(By.XPATH,".//h3[contains(text(), 'Job type')]").find_element(By.XPATH, "following-sibling::div").find_elements(By.XPATH, "./ul/li")
                        print(len(children))
                        for element in children:
                            jobType.append(driver.execute_script("return arguments[0].innerText;",element))
                        # print(F"Shift and schedule: {shiftAndSchedule}")
                    except Exception as e:
                        # print("No shift and schedule found")
                        jobType= pattern_jobtype.findall(salaryInfoAndJobType)
                    
                print(F"Salary: {salary}")
                print(F"JOB type: {jobType}")
                try:
                    
                    shiftAndSchedule = []
                    salaryInfoAndJobType_children=positionsDetailsElement.find_element(By.XPATH,".//h3[contains(text(), 'Quart de travail et horaire')]").find_element(By.XPATH, "following-sibling::div").find_elements(By.XPATH, "./ul/li")
                    for element in salaryInfoAndJobType_children:
                        shiftAndSchedule.append(driver.execute_script("return arguments[0].innerText;",element))
                    print(F"Shift and schedule: {shiftAndSchedule}")
                except Exception as e:
                    # print("No shift and schedule found")
                    shiftAndSchedule=''
                if shiftAndSchedule =='':
                    try:
                        shiftAndSchedule = []
                        salaryInfoAndJobType_children=positionsDetailsElement.find_element(By.XPATH,".//h3[contains(text(), 'Shift and schedule')]").find_element(By.XPATH, "following-sibling::div").find_elements(By.XPATH, "./ul/li")
                        for element in salaryInfoAndJobType_children:
                            shiftAndSchedule.append(driver.execute_script("return arguments[0].innerText;",element))
                        print(F"Shift and schedule: {shiftAndSchedule}")
                    except Exception as e:
                        # print("No shift and schedule found")
                        shiftAndSchedule=''
                jobBenefits = []
                try:
                    benefits = positionsDetailsElement.find_elements(By.XPATH, ".//div[@id='benefits']/div/div/span/ul[@class='css-8tnble eu4oa1w0']/li")
                    for benefit in benefits:
                        jobBenefits.append(driver.execute_script("return arguments[0].innerText;",benefit))
                
                except Exception as e:
                    # print("No benefits found")
                    benefits=''
                    
                try:
                    description = driver.execute_script("return arguments[0].innerText;",positionsDetailsElement.find_element(By.XPATH, ".//div[@id='jobDescriptionText']"))
                except Exception as e:
                    # print("No description found")
                    description=''
        
    positionsDetails["positionTitle"]=positionTitle
    positionsDetails["companyName"]=companyName
    positionsDetails["location"]=location
    positionsDetails["salaryInfoAndJobType"]=salaryInfoAndJobType 
    positionsDetails["description"]=description
    positionsDetails["datePosted"]=positionPostedDate
    # positionsDetails["datePosted"]=positionPostedDate
    positionsDetails["salary"]=salary
    positionsDetails["jobType"]=jobType
    positionsDetails["shiftAndSchedule"]=shiftAndSchedule
    positionsDetails["jobBenefits"]=jobBenefits
    positionsDetails["updatedDate"]= (datetime.now()).strftime('%Y-%m-%d')
    if country not in positionsDetailsList:
        positionsDetailsList[country]={}
    if profession not in positionsDetailsList[country]:
        positionsDetailsList[country][profession]={}
    if language not in positionsDetailsList[country][profession]:
        positionsDetailsList[country][profession][language]=[]
    positionsDetailsList[country][profession][language].append(positionsDetails)
    time.sleep(2)

In [25]:
professions_by_field=mongo_manager.read_all_documents_in_collection(collection_names[0])
professions=set() 
for key in professions_by_field.keys():
    professions.update(professions_by_field[key]["professions"])

# clean_professions = []
# df_professsions = pd.DataFrame(professions)
# for profession in df_professsions.drop_duplicates().to_numpy().tolist():
#     clean_professions.append(profession[0])
    
professions=list(professions)
professions.sort()
print(professions)
print(len(professions)) 

['Accompagnatrice spirituelle ou accompagnateur spirituel', 'Accompagnatrice, accompagnateur, musicienne ou musicien dans de petits ensembles', 'Acheteuse ou acheteur', 'Actuaire', 'Adjointe ou adjoint à la recherche', 'Administratrice ou administrateur', 'Administratrice ou administrateur de contrats', 'Administratrice ou administrateur de programmes de développement durable', 'Administratrice ou administrateur de prêts commerciaux', 'Affichiste', 'Agent ou agente d’approvisionnement et de gestion contractuelle', 'Agente commerciale ou agent commercial, politique ou du service extérieur', 'Agente culturelle ou agent culturel', 'Agente culturelle ou agent culturel / Cultural agent', 'Agente et agent de recherche et de planification socioéconomique', 'Agente ou agent (de programmes, de recherche en planification socioéconomique)', 'Agente ou agent au développement des affaires', 'Agente ou agent communautaire, de développement, de planification, des politiques sociales, de programmation

In [26]:
countries=["Canada","US","France"]
languages=["fr","en"]

In [27]:
import random
from selenium.webdriver.chrome.options import Options


USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) Gecko/20100101 Firefox/124.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.2420.81",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 OPR/109.0.0.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14.4; rv:124.0) Gecko/20100101 Firefox/124.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_4_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4.1 Safari/605.1.15",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 OPR/109.0.0.0",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Linux i686; rv:124.0) Gecko/20100101 Firefox/124.0",
]

# def get_random_proxy():
#     return random.choice(PROXIES)


chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument(f'user-agent={random.choice(USER_AGENTS)}')


def random_sleep(min_time=1, max_time=5):
    time.sleep(random.uniform(min_time, max_time))

def indeedDataFill(ulavalPosition, location,page_lang):
    if location=="Canada":
        driver.get(f"https://emplois.ca.indeed.com/jobs?q={ulavalPosition.replace(' ','+')}&l={location.replace(' ','+')}&hl={page_lang}")
    if location=="US":
        driver.get(f"https://www.indeed.com/jobs?q={ulavalPosition.replace(' ','+')}&l={location.replace(' ','+')}&hl={page_lang}")
    if location=="France":
        driver.get(f"https://www.indeed.fr/jobs?q={ulavalPosition.replace(' ','+')}&l={location .replace(' ','+')}&hl={page_lang}")    
    random_sleep()
    # try:
    #     title_input = driver.find_element(By.ID, 'text-input-what')
    #     location_input = driver.find_element(By.ID, 'text-input-where')
    # except Exception as e:
    #     e
    # if title_input:
    #     driver.execute_script("arguments[0].value = '';", title_input)
    #     # driver.execute_script("arguments[0].value = '';", location_input)
    #     # location_input.send_keys(location)
    #     title_input.send_keys(ulavalPosition, Keys.RETURN)
    # return title_input, location_input
    

In [28]:
import requests

def translate_text(text, target_language):
    url = "https://api.mymemory.translated.net/get"
    params = {
        'q': text,
        'langpair': f'fr|{target_language}'  # Exemple pour traduire du français vers l'anglais
    }
    response = requests.get(url, params=params)
    return response.json().get('responseData', {}).get('translatedText')

def traduction(profession, language):
    if language=="fr":
        return profession
    if language=="en":
        return translate_text(profession, "en")
    
traduction("Infirmier", "en")

'Nurse'

In [29]:
def get_line_of_profession(lines,profession):
    index=-1
    for line_number, line in enumerate(lines):
        if profession in line:
            index=line_number
            break
    return index

def delete_last_index_str(line,str_index):
    return line.replace(str_index, "")

In [ ]:

line_idx = -1

# Open the progress file and read the lines
with open("data_retriever_progress.txt", 'r', encoding='utf-8') as data_retriever_progress_file:
    lines = data_retriever_progress_file.readlines()

# Ensure lines list is initialized
if not lines:
    lines = []

try:
    # Initialize the WebDriver
    driver = webdriver.Chrome(options=chrome_options)

    # Iterate over the list of professions, starting from index 12
    for i, profession in tqdm(enumerate(professions[13:14], start=13), desc="Professions", leave=True):
        print(profession)
        
        # Construct the line entry for the current profession
        line = profession + f" [{i}] "
        line_idx = get_line_of_profession(lines, profession)
        print(line_idx)
        
        # Check if the profession has already been processed
        if line_idx != -1:
            message = "On a remarqué que vous avez déjà récupéré les postes de cette profession "
            message += "en totalité" if "(end)" in lines[line_idx] else "mais en partie"
            message += ". Est-ce que vous voulez recommencer la récupération ? Y/N "
            
            user_input = input(message)
            while user_input.lower() not in "y/n":
                user_input = input(message)
            
            if user_input.lower() == "n":
                continue
        
        # Iterate over countries
        for j, country in tqdm(enumerate(countries, start=0), desc="Countries", leave=True):
            line = delete_last_index_str(line, "cntryIdx" + f" [{j - 1}] ")
            line += "cntryIdx" + f" [{j}] "

            # Iterate over languages
            for k, language in tqdm(enumerate(languages, start=0), desc="Languages", leave=True):
                line = delete_last_index_str(line, "langIdx" + f" [{k - 1}] ")
                line += "langIdx" + f" [{k}] "

                # Iterate over page languages
                for l, page_lang in tqdm(enumerate(languages, start=0), desc="Page Languages", leave=True):
                    line = delete_last_index_str(line, "pgLangIdx" + f" [{l - 1}] ")
                    line += "pgLangIdx" + f" [{l}] "

                    # Retrieve Indeed data
                    indeedDataFill(traduction(profession, language), country, page_lang)
                    
                    # Check for the presence of the "Next" button for pagination
                    try:
                        pagination_next_element = driver.find_element(By.XPATH, "//nav/ul/li/a[@data-testid='pagination-page-next']")
                    except Exception:
                        pagination_next_element = None
                    
                    # Process each page until there's no "Next" button
                    if not pagination_next_element:
                        print(f"Button {pagination_next_element}")
                        indeedPositionInfos(profession, country, language, page_lang)
                    
                    while pagination_next_element:
                        indeedPositionInfos(profession, country, language, page_lang)
                        driver.execute_script("arguments[0].click();", pagination_next_element)
                        
                        try:
                            pagination_next_element = driver.find_element(By.XPATH, "//nav/ul/li/a[@data-testid='pagination-page-next']")
                        except Exception:
                            pagination_next_element = None
        
        # Update or add the line entry in the progress file
        if line_idx == -1:
            lines.append(line + " (end)\n")
        else:
            lines[line_idx] = line + " (end)\n"
    
    driver.close()

except Exception as e:
    print(e)
    driver.close()

print("Ligne " + line)
print(line_idx)

# Check if line entry is already in the progress list, add it if not
if line not in lines:
    if "(end)" not in lines:
        lines.append(line)

# Write the updated lines back to the progress file
with open("data_retriever_progress.txt", 'w', encoding='utf-8') as data_retriever_progress_file:
    data_retriever_progress_file.writelines(lines)


Professions: 0it [00:00, ?it/s]

Agente culturelle ou agent culturel / Cultural agent
-1


Countries: 0it [00:00, ?it/s]

Languages: 0it [00:00, ?it/s]

Page Languages: 0it [00:00, ?it/s]

Button None
Button None


Page Languages: 0it [00:00, ?it/s]

Button None


In [ ]:
len(positionsDetailsList)

0

In [ ]:
if mongo_manager.get_if_database_is_available_for_adding_data(positionsDetailsList):
    for country in tqdm(positionsDetailsList.keys()):
        for profession in tqdm(positionsDetailsList[country].keys()):
            try:
                mongo_manager.update_data_before_save_new_json_with_GridFS(country,{profession:positionsDetailsList[country][profession]})
            except Exception as e:
                print(e)
else:
    mongo_manager_2 = MongoDBManager(os.getenv('MONGODB_URI_2'), database_name)
    if mongo_manager_2.get_if_database_is_available_for_adding_data(positionsDetailsList):
        for country in tqdm(positionsDetailsList.keys()):
            for profession in tqdm(positionsDetailsList[country].keys()):
                try:
                    mongo_manager_2.update_data_before_save_new_json_with_GridFS(country,{profession:positionsDetailsList[country][profession]})
                except Exception as e:
                    print(e)
    else:
        print("No database available for adding data")

# Cuisine interne de betch :)

In [ ]:
data=mongo_manager.read_all_documents_in_collection_with_GridFS()

In [ ]:
mongo_manager.find_and_solve_error()

Find and solve error


In [ ]:
import json
positionsDetailsList=json.load(open('temp_data copy.json'))

In [ ]:
mongo_manager.gettotalsizeof(positionsDetailsList)/1024/1024

Dictionary saved in temp_data.json
File size: 44488542 bytes


42.42757987976074

In [ ]:
mongo_manager.get_if_database_is_available_for_adding_data(positionsDetailsList)

Taille des données existante : 16.92078399658203
Taille des données à ajouter : 0.00017547607421875
Taille restante Estimée après ajout de la nouvelle donnée: 495.07904052734375
Database is available to add data


True

In [ ]:
def data_recurcive_length(data):
    try:
        for key in data.keys():
            for profession in data[key].keys():
                for language in data[key][profession].keys():
                    print(f"{key} - {profession} - {language} : {len(data[key][profession][language])}")
    except Exception as e:
        print(e)
        print(data[key][profession])

In [ ]:
data_recurcive_length(positionsDetailsList)

US - Accompagnatrice spirituelle ou accompagnateur spirituel - en : 36
US - Accompagnatrice, accompagnateur, musicienne ou musicien dans de petits ensembles - fr : 15
US - Accompagnatrice, accompagnateur, musicienne ou musicien dans de petits ensembles - en : 1530
France - Accompagnatrice spirituelle ou accompagnateur spirituel - fr : 8
France - Accompagnatrice, accompagnateur, musicienne ou musicien dans de petits ensembles - fr : 972
France - Accompagnatrice, accompagnateur, musicienne ou musicien dans de petits ensembles - en : 36
Canada - Accompagnatrice, accompagnateur, musicienne ou musicien dans de petits ensembles - fr : 46
Canada - Accompagnatrice, accompagnateur, musicienne ou musicien dans de petits ensembles - en : 460
Canada - Acheteuse ou acheteur - fr : 271
Canada - Acheteuse ou acheteur - en : 967


In [ ]:
data_recurcive_length(data)

Canada - Agente ou agent communautaire, de développement, de planification, des politiques sociales, de programmation, de recherche - fr : 13
Canada - Accompagnatrice spirituelle ou accompagnateur spirituel - fr : 101
Canada - Accompagnatrice spirituelle ou accompagnateur spirituel - en : 23
Canada - Accompagnatrice, accompagnateur, musicienne ou musicien dans de petits ensembles - fr : 179
Canada - Accompagnatrice, accompagnateur, musicienne ou musicien dans de petits ensembles - en : 1040
Canada - Actuaire - fr : 119
Canada - Actuaire - en : 609
Canada - Acheteuse ou acheteur - fr : 634
Canada - Acheteuse ou acheteur - en : 1212
Canada - Adjointe ou adjoint à la recherche - fr : 705
Canada - Adjointe ou adjoint à la recherche - en : 907
Canada - Administratrice ou administrateur - fr : 518
Canada - Administratrice ou administrateur - en : 1398
Canada - Administratrice ou administrateur de contrats - fr : 128
Canada - Administratrice ou administrateur de contrats - en : 1104
Canada - 

In [ ]:
update_data=json.load(open('data.json'))
for country in data:
    if country not in update_data:
        update_data[country] = {}
    for key, nested_data in data[country].items():
        if key not in countries:
            if key not in update_data[country]:
                update_data[country][key] = {}
            for key2, values in nested_data.items():
                print(f"{country} - {key} - {key2}")
                if key2 in update_data[country][key] and len(update_data[country][key][key2]) == len(values):
                    print("Nothing to update")
                    continue
                if key2 in update_data[country][key]:
                    update_data[country][key][key2].extend(values)
                else:
                    update_data[country][key][key2] = values

Canada - Analyste financière ou analyste financier - fr
Canada - Analyste de marché - fr
Canada - Analyste de marché - en
Canada - Trésorière, trésorier ou gestionnaire de la trésorerie - fr
Canada - Trésorière, trésorier ou gestionnaire de la trésorerie - en
US - Analyste de marché - fr
US - Analyste de marché - en
US - Trésorière, trésorier ou gestionnaire de la trésorerie - fr
Nothing to update
US - Trésorière, trésorier ou gestionnaire de la trésorerie - en
France - Trésorière, trésorier ou gestionnaire de la trésorerie - fr
Nothing to update
France - Trésorière, trésorier ou gestionnaire de la trésorerie - en
Nothing to update


In [ ]:
data_recurcive_length(update_data)

Canada - Analyste financière ou analyste financier - fr : 690
Canada - Analyste financière ou analyste financier - en : 1081
Canada - Trésorière, trésorier ou gestionnaire de la trésorerie - fr : 437
Canada - Trésorière, trésorier ou gestionnaire de la trésorerie - en : 3687
Canada - Analyste de marché - fr : 391
Canada - Analyste de marché - en : 1058
US - Trésorière, trésorier ou gestionnaire de la trésorerie - fr : 7
US - Trésorière, trésorier ou gestionnaire de la trésorerie - en : 3962
US - Analyste de marché - fr : 7
US - Analyste de marché - en : 720
France - Trésorière, trésorier ou gestionnaire de la trésorerie - fr : 306
France - Trésorière, trésorier ou gestionnaire de la trésorerie - en : 198


In [ ]:
for country in tqdm(update_data.keys()):
    for profession in tqdm(update_data[country].keys()):
        try:
            mongo_manager.update_data_before_save_new_json_with_GridFS(country,{profession:update_data[country][profession]})
        except Exception as e:
            print(e)

Countries:   0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

dict_keys([])
Data added to Canada with file_id: 66c0fdd05f22bec961a5d6ef

dict_keys(['Analyste financière ou analyste financier'])
Data added to Canada with file_id: 66c0fddf5f22bec961a5d717

dict_keys(['Analyste financière ou analyste financier', 'Trésorière, trésorier ou gestionnaire de la trésorerie'])
Data added to Canada with file_id: 66c0fe1a5f22bec961a5d793



Languages:   0%|          | 0/2 [00:00<?, ?it/s]

dict_keys([])
Data added to US with file_id: 66c0fe2f5f22bec961a5d833

dict_keys(['Trésorière, trésorier ou gestionnaire de la trésorerie'])
Data added to US with file_id: 66c0fe495f22bec961a5d88f



  0%|          | 0/1 [00:00<?, ?it/s]

dict_keys([])
Data added to France with file_id: 66c0fe525f22bec961a5d8fa



In [ ]:
db_data=mongo_manager.read_all_documents_in_collection_with_GridFS()
# with open('data.json', 'w') as fichier:
#     json.dump(positionsDetailsList, fichier)

KeyboardInterrupt: 

In [ ]:
db_data[:10]

NameError: name 'db_data' is not defined

In [ ]:
data_recurcive_length(db_data)

Canada - Analyste financière ou analyste financier - fr : 690
Canada - Analyste financière ou analyste financier - en : 1081
Canada - Trésorière, trésorier ou gestionnaire de la trésorerie - fr : 437
Canada - Trésorière, trésorier ou gestionnaire de la trésorerie - en : 3687
Canada - Analyste de marché - fr : 1426
Canada - Analyste de marché - en : 5204
Canada - Actuaire - fr : 115
Canada - Actuaire - en : 851
US - Trésorière, trésorier ou gestionnaire de la trésorerie - fr : 7
US - Trésorière, trésorier ou gestionnaire de la trésorerie - en : 3962
US - Analyste de marché - fr : 36
US - Analyste de marché - en : 2791
US - Actuaire - en : 1287
US - Experte ou expert en science des données (data scientist) - fr : 7
US - Experte ou expert en science des données (data scientist) - en : 774
France - Trésorière, trésorier ou gestionnaire de la trésorerie - fr : 306
France - Trésorière, trésorier ou gestionnaire de la trésorerie - en : 198
France - Actuaire - fr : 684
France - Actuaire - en :

In [ ]:

# for country in tqdm(positionsDetailsList.keys(), desc="Countries"):
#     for profession in tqdm(positionsDetailsList[country].keys(), desc="Professions", leave=False):
#         for language in tqdm(positionsDetailsList[country][profession].keys(), desc="Languages", leave=False):
#             document_id = country
#             collection_name = collection_names[2] 
#             try:
#                 collection = mongo_manager.db[collection_name]
#                 document = collection.find_one(
#                 {
#                     "_id": document_id, 
#                     f"{profession}.{language}": {"$exists": True} 
#                 })
#                 print(document)
                
#                 if document:
#                     old_data= document[profession][language]
#                     new_data=positionsDetailsList[country][profession][language]
#                     if len(old_data) == len(new_data):
#                         print("Nothing to update for", country, profession, language)
#                     else:
#                        update_data = {f"{country}.{profession}.{language}":old_data.extend(new_data) }  
#                 else:
#                     update_data = {f"{country}.{profession}.{language}":positionsDetailsList[country][profession][language] } 

#                 collection.update_one({'_id': document_id}, {'$set': update_data})
#                 print(f'Data updated in {collection_name}/{document_id}')

#             except Exception as e:
#                 print(f'Error updating data in MongoDB: {e}')


In [ ]:
files_ids =     mongo_manager.db.fs.chunks.distinct("files_id")
files_ids


[ObjectId('66d699ac943cfa0276f23855'),
 ObjectId('66d69aa0604efd403dc468a4'),
 ObjectId('66d69bc0943cfa0276f23ca1')]

Upserted file: {'_id': ObjectId('66d699ac943cfa0276f23855'), 'filename': 'reconstructed_file_66d699ac943cfa0276f23855', 'chunkSize': 261120, 'length': 192095142, 'uploadDate': datetime.datetime(2024, 9, 3, 5, 7, 57, tzinfo=<bson.tz_util.FixedOffset object at 0x0000024CA04087D0>)}
Reconstituted file: {'_id': ObjectId('66d699ac943cfa0276f23855'), 'filename': 'reconstructed_file_66d699ac943cfa0276f23855', 'chunkSize': 261120, 'length': 192095142, 'uploadDate': datetime.datetime(2024, 9, 3, 5, 7, 57, tzinfo=<bson.tz_util.FixedOffset object at 0x0000024CA04087D0>)}
Upserted file: {'_id': ObjectId('66d69aa0604efd403dc468a4'), 'filename': 'reconstructed_file_66d69aa0604efd403dc468a4', 'chunkSize': 261120, 'length': 67368960, 'uploadDate': datetime.datetime(2024, 9, 3, 5, 12, tzinfo=<bson.tz_util.FixedOffset object at 0x0000024CA04087D0>)}
Reconstituted file: {'_id': ObjectId('66d69aa0604efd403dc468a4'), 'filename': 'reconstructed_file_66d69aa0604efd403dc468a4', 'chunkSize': 261120, 'length': 